In [76]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
np.random.seed(0)

In [77]:
data = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")

In [78]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
data = np.array(data)
test = np.array(test)
m,n = data.shape
labels = data[:,0]
values = data[:,1:n]
test_labels = test[:,0]
test_values = test[:,1:n]

In [146]:

class Layer:
    def __init__(self):
        self.inputNum = None
        self.outputNum = None

class Dense(Layer):
    def __init__(self,n_inputs,n_neurons):
        self.weights = 0.1*np.random.rand(n_neurons,n_inputs)
        self.bias = np.zeros((n_neurons,1))

    def foward(self, input):
        self.input = input
        return np.dot(self.weights,self.input) + self.bias
    
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient,self.input.T)
        self.weights -= learning_rate*weights_gradient
        self.bias -= learning_rate*output_gradient
        return np.dot(self.weights.T,output_gradient)
    

class NeuralNetwork():
    def __init__(self,layers:list[Layer])->list[int]:
        self.layers = layers

    def cross_entropy(self, y_true, y_pred):
        return -np.sum(y_true * np.log(y_pred))

    def cross_entropy_prime(self, y_true, y_pred):      
        return - (y_true / y_pred)

    
    def verifyAnswer(self,answer,prediction):
        return prediction.argmax() == answer
    
    def oneHotEncode(self,value):
        vector = np.zeros((10, 1))
        vector[value, 0] = 1
        return vector

    def calculateError(self,predictedVector,correctVector):
        return predictedVector-correctVector



    def batch_train(self,dataset_values,dataset_labels, epochs, learningRate):
        for epoch in range(epochs):
            correct = 0
            for img,answer in zip(dataset_values,dataset_labels):
                desiredOutput = self.oneHotEncode(answer)
                currentImg = np.reshape(img,(n-1,1))
                input = currentImg
                for i,layer in enumerate(self.layers):
                    input = layer.foward(input)
                    # print(f"Camada numero {i}: {input}")
                correct += 1 if self.verifyAnswer(answer,input) else 0
                grad = self.calculateError(input,desiredOutput)
            for layer in reversed(self.layers):
                grad = layer.backward(grad,learningRate)
            print(f"Epoch {epoch} --> Accuracy = {correct/len(dataset_labels)*100:.2f}%")

In [157]:
class Activation(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def foward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

class Relu(Activation):
    def __init__(self):
        def function(inputs):
            return np.maximum(0,inputs)

        def function_prime(inputs):
            return inputs > 0
        super().__init__(function,function_prime)


class Softmax(Activation):
    def __init__(self):
        def function(input):
            tmp = np.exp(input-np.max(input))
            self.output = tmp/np.sum(tmp)
            return self.output

        def function_prime(inputs):
            n = np.size(self.output)
            tmp = np.tile(self.output,n)
            return tmp*(np.identity(n)-np.transpose(tmp))
        super().__init__(function,function_prime)


class Loss:
    def calculate(self,output,y):
        sample_losses = self.foward(output,y)

class CategoricalCrossEntropy(Loss):
    def foward(self,y_pred,y_true):
        y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)
        return -np.log(np.sum(y_pred_clipped*y_true))


In [158]:

network = NeuralNetwork([
    Dense(784,10),
    Relu(),
    Dense(10,10),
    Relu(),
])

In [159]:
network.batch_train(values,labels,epochs=20,learningRate=0.1)

Epoch 0 --> Accuracy = 7.94%
Epoch 1 --> Accuracy = 9.87%
Epoch 2 --> Accuracy = 9.87%
Epoch 3 --> Accuracy = 9.87%
Epoch 4 --> Accuracy = 9.87%
Epoch 5 --> Accuracy = 9.87%
Epoch 6 --> Accuracy = 9.87%
Epoch 7 --> Accuracy = 9.87%
Epoch 8 --> Accuracy = 9.87%
Epoch 9 --> Accuracy = 9.87%
Epoch 10 --> Accuracy = 9.87%


KeyboardInterrupt: 

In [9]:
correct = 0
for num in range(len(test_labels)):
    input = np.reshape(values[num],(n-1,1))
    for layer in network.layers:
        input = layer.foward(input)
    if input.argmax() == labels[num]:
        correct+=1
print(f"porcentagem de acerto: {correct/len(test_labels)*100:.6f}%")

porcentagem de acerto: 10.010000%


In [99]:
# class Layer:
#     def __init__(self):
#         self.input = None
#         self.output = None

#     def foward(self,input):
#         pass

#     def backward(self,output_gradient,learning_rate):
#         pass

# class Dense(Layer):
#     def __init__(self,input_size,output_size):
#         self.weights = np.random.randn(output_size,input_size)
#         self.bias = np.random.randn(output_size,1)

#     def foward(self, input):
#         self.input = input
#         return np.dot(self.weights,self.input) + self.bias
    
#     def backward(self, output_gradient, learning_rate):
#         weights_gradient = np.dot(output_gradient,self.input.T)
#         self.weights -= learning_rate*weights_gradient
#         self.bias -= learning_rate*output_gradient
#         return np.dot(self.weights.T,output_gradient)

class Activation(Layer):
    def __init__(self,activation,acticvation_prime):
        self.activation = activation
        self.activation_prime = acticvation_prime

    def foward(self, input):
        self.input = input
        return self.activation(self.input)
    
    def backward(self,output_gradient, learningRate):
        return np.multiply(output_gradient,self.activation_prime(self.input))


class Tanh(Activation):
    def __init__(self):
        tanh = lambda x: np.tanh(x)
        tanh_prime = lambda x: 1- np.tanh(x)**2
        super().__init__(tanh,tanh_prime)

In [69]:
layer = Dense(3,3)

layer.foward(np.array([[0.4],[0.3],[0.2]]))

array([[ 0.05700469],
       [ 0.05902258],
       [-0.01314645]])

In [75]:
0.1*np.random.randn(3,4)

array([[-0.05895288, -0.09275179,  0.16710183,  0.20050953],
       [ 0.12064281,  0.08703798,  0.05213443, -0.16184937],
       [-0.15554611,  0.12143441,  0.07207063, -0.02092479]])